In [165]:
# Import dependencies 
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import apache_beam as beam

# Print version for easy debugging 
print "Library versions \n Tensorflow version: {} \n DataFlow version: {}".format(tf.__version__, beam.__version__)

Library versions 
 Tensorflow version: 1.11.0 
 DataFlow version: 2.10.0


<h2>Setup environment variables</h2>

Store local paths and filenames in environment variables for easy use. 

In [167]:
import os

# Store the root directory of the project
CWD = os.getcwd() # path to this notebook on the local filesystem
ROOT,_ = os.path.split(CWD) # on level up, the root directory of the project 

# Save path to raw in envron variables  
DATA_DIR = os.path.join(ROOT,'raw_data/')
DATA_FILE_NAME = 'true_car_listings.csv'
STATS_FILE_NAME = 'stats.tfrecord'

DATA_PATH = os.path.join(DATA_DIR,DATA_FILE_NAME)
STATS_PATH = os.path.join(DATA_DIR,STATS_FILE_NAME) # path to store statistics 

# Check on the root oflder
print "Root project folder is: {}".format(ROOT)

Root project folder is: /Users/evanderknaap/Documents/Projects/tfvalidate


<h2>Load the data into memory</h2>

<h2>Compute statistics on data on local machine</h2>

Load the tfdv dependencies, this may take some time 

In [156]:
# Import tensorflow
import tensorflow_data_validation as tfdv

print "TFDV version: {}".format(tfdv.version.__version__)

TFDV version: 0.11.0


Next, we point TFDV to the location of our raw data, and compute the statistics on our local machine using an apache beam pipeline. They will be stored as a protobuffer .tfrecord file in the the folder located in the statistics path. We'll get a warning if there is an existing .tfrecord file, which you can choose to overwrite.

In [154]:
# Compute statistics frmo the raw data and store stats as a tfrecord
tfdv.generate_statistics_from_csv(data_location=DATA_PATH, output_path=STATS_PATH)

datasets {
  num_examples: 852122
  features {
    name: "City"
    type: STRING
    string_stats {
      common_stats {
        num_non_missing: 852122
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 85212.2
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 85212.2
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 85212.2
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 85212.2
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 85212.2
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 85212.2
          }
          